In [1]:
#importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as s
import math
import datetime as dt
from dateutil.relativedelta import relativedelta
import warnings

In [2]:
warnings.filterwarnings('ignore')
#read data 
cus_dem = pd.read_csv('Customer_demographic.csv',usecols=lambda x: x != 'default')
cus_add = pd.read_csv('Customer_address.csv')
cus_txn = pd.read_csv('Transaction.csv')
#Covid_Data.rename(columns={'Difference_Yesterday':'PCR_Postive_Count','Hospitalized_diff':'PCR_Positive_Hospitalised','Death_diff':'Deceased'}, inplace = True)


In [3]:
#cus_dem.head(10)
#cus_add.head(10)
#cus_txn.head(10)

In [4]:
cus_dem['DOB']=pd.to_datetime(cus_dem['DOB'])
#cus_dem['Age'] = [relativedelta(pd.to_datetime('now'), d).years for d in cus_dem['DOB']]
now = pd.to_datetime('now')
cus_dem['Age'] = (now.year - cus_dem['DOB'].dt.year) - ((now.month - cus_dem['DOB'].dt.month) < 0)
#cus_dem['Age'] = cus_dem["DOB"].apply(lambda x : (pd.datetime.now().year - x.year))
cus_dem['Age'] = cus_dem['Age'].fillna(0)
cus_dem['Age'] = cus_dem['Age'].astype(int)
print(cus_dem.dtypes)

customer_id                                     int64
first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                        float64
Age                                             int32
dtype: object


In [5]:
cus_dem.to_csv('cus_dem_age.csv')

In [6]:
cus_dem_txn = cus_dem.merge(cus_txn, on="customer_id", how='inner')

In [7]:
cus_dem_txn_filtered = cus_dem_txn[cus_dem_txn['brand'].notna()]

In [8]:
"""
cus_dem_txn_filtered['brand_int'] = 0
cus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Solex')] = 1
cus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Trek Bicycles')] = 2
cus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('OHM Cycles')] = 3
cus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Norco Bicycles')] = 4
cus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Giant Bicycles')] = 5
cus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('WeareA2B')] = 6
#cus_dem_txn_filtered['brand_int'] = df['brand'].str.contains('a').astype(int)
"""

"\ncus_dem_txn_filtered['brand_int'] = 0\ncus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Solex')] = 1\ncus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Trek Bicycles')] = 2\ncus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('OHM Cycles')] = 3\ncus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Norco Bicycles')] = 4\ncus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('Giant Bicycles')] = 5\ncus_dem_txn_filtered['brand_int'][cus_dem_txn_filtered['brand'].astype(str).str.contains('WeareA2B')] = 6\n#cus_dem_txn_filtered['brand_int'] = df['brand'].str.contains('a').astype(int)\n"

In [9]:
cus_dem_txn_filtered = cus_dem_txn_filtered[cus_dem_txn_filtered['Age']<101]
#cus_dem_txn_filtered

In [10]:
"""
plt.bar(cus_dem_txn_filtered['Age'],cus_dem_txn_filtered['brand_int'])
plt.title('Age v/s Brand')
plt.xlabel('Brand')
plt.ylabel('Age')
plt.show()
"""

"\nplt.bar(cus_dem_txn_filtered['Age'],cus_dem_txn_filtered['brand_int'])\nplt.title('Age v/s Brand')\nplt.xlabel('Brand')\nplt.ylabel('Age')\nplt.show()\n"

In [11]:
age_groups = pd.cut(cus_dem_txn_filtered['Age'], bins=[0,1,20, 40, 60,100, np.inf])

In [12]:
df_agegrp_brand = pd.crosstab(age_groups, cus_dem_txn_filtered['brand'])
df_agegrp_brand

brand,Giant Bicycles,Norco Bicycles,OHM Cycles,Solex,Trek Bicycles,WeareA2B
Age,,,,,,
"(1.0, 20.0]",14,11,16,16,8,12
"(20.0, 40.0]",1054,939,1024,1428,993,1065
"(40.0, 60.0]",1708,1506,1548,2182,1533,1726
"(60.0, 100.0]",464,375,391,549,387,405


In [13]:
df_agegrp_txncnt = pd.crosstab(age_groups, cus_dem_txn_filtered['transaction_id'].count(), colnames=['Transaction Count'])
df_agegrp_txncnt

Transaction Count,19792
Age,
"(1.0, 20.0]",77
"(20.0, 40.0]",6503
"(40.0, 60.0]",10203
"(60.0, 100.0]",2571


In [14]:
df_agegrp_gender = pd.crosstab(age_groups, cus_dem_txn_filtered['gender'])
df_agegrp_gender

gender,F,M
Age,,
"(1.0, 20.0]",40,37
"(20.0, 40.0]",3249,3254
"(40.0, 60.0]",5358,4845
"(60.0, 100.0]",1274,1297


In [15]:
cus_add_txn = cus_add.merge(cus_txn, on="customer_id", how='inner')
#cus_add_txn

In [16]:
df_state_brand = pd.crosstab(cus_add_txn['state'], cus_add_txn['brand'])
df_state_brand

brand,Giant Bicycles,Norco Bicycles,OHM Cycles,Solex,Trek Bicycles,WeareA2B
state,,,,,,
NSW,1735,1587,1663,2249,1583,1746
QLD,717,628,652,902,613,710
VIC,856,690,724,1097,787,834
